In [53]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
API_KEY = '2qBcnThLnlearCE77ZchoF2uJLQ'  # Replace with your actual API key
SINCE_DATE = int(datetime(2024, 1, 17).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/institutions/us_spot_etf_flows_net'
]

# Dictionary for stacked metrics: URL -> key to extract
METRICS_STACKED = {
    'https://api.glassnode.com/v1/metrics/institutions/us_spot_etf_balances_all': ['total','GBTC'],
    'https://api.glassnode.com/v1/metrics/institutions/us_spot_etf_flows_all': ['total','GBTC']
}

def fetch_glassnode_data(url, asset='BTC', key_to_extract=None):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'c': 'USD'
    }
    
    metric_base_name = url.split('/')[-1]
    
    # Set format based on whether it's a stacked metric
    if url in METRICS_STACKED:
        params['f'] = 'JSON'
    else:
        params['f'] = 'CSV'

    response = requests.get(url, params=params)
    if response.status_code == 200:
        # Handle stacked metric response
        if url in METRICS_STACKED and key_to_extract:
            data = response.json()
            # Create column name combining metric name and the specific key
            column_name = f"{metric_base_name}_{key_to_extract}"
            
            # Extract timestamps and the specific key we want
            timestamps = [entry['t'] for entry in data]
            values = [entry['o'][key_to_extract] for entry in data]
            
            df = pd.DataFrame({
                't': timestamps,
                column_name: values
            })
        else:
            # Handle regular metric response
            df = pd.read_csv(io.StringIO(response.text))
            # Keep original column names from CSV
            if len(df.columns) != 2:
                print(f"Unexpected number of columns in response from {url}: {df.columns}")
                return None
            df.columns = ['t', metric_base_name]
        
        df['t'] = pd.to_datetime(df['t'], unit='s')
        # Convert the metric column to numeric, excluding the timestamp column
        for col in df.columns:
            if col != 't':
                df[col] = pd.to_numeric(df[col], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
print("Fetching price data...")
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]

# Fetch regular metrics
print("Fetching regular metrics...")
for metric_url in METRICS:
    print(f"Fetching {metric_url.split('/')[-1]}...")
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

# Fetch stacked metrics
print("Fetching stacked metrics...")
for metric_url, keys in METRICS_STACKED.items():
    # Handle both single key (string) and multiple keys (list) cases
    if isinstance(keys, str):
        keys = [keys]  # Convert single key to list
    for key in keys:
        print(f"Fetching {metric_url.split('/')[-1]} - {key}...")
        metric_df = fetch_glassnode_data(metric_url, key_to_extract=key)
        if metric_df is not None:
            all_dfs.append(metric_df)

print("Merging all dataframes...")
merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

print("Final columns in merged_df:", merged_df.columns.tolist())

Fetching price data...
Fetching regular metrics...
Fetching us_spot_etf_flows_net...
Fetching stacked metrics...
Fetching us_spot_etf_balances_all - total...
Fetching us_spot_etf_balances_all - GBTC...
Fetching us_spot_etf_flows_all - total...
Fetching us_spot_etf_flows_all - GBTC...
Merging all dataframes...
Final columns in merged_df: ['price_usd_close', 'us_spot_etf_flows_net', 'us_spot_etf_balances_all_total', 'us_spot_etf_balances_all_GBTC', 'us_spot_etf_flows_all_total', 'us_spot_etf_flows_all_GBTC']


In [54]:
import yfinance as yf
from datetime import date, datetime, time, timedelta

# [Previous code remains the same until the final print statement]
print("Final columns in merged_df:", merged_df.columns.tolist())

def stock_ticker(ticker):
    # Download historical financial data
    response = yf.download(ticker, start="2020-01-17", end=datetime.combine(date.today(), time()))
    response['date'] = pd.to_datetime(response.index,utc=True)
    response = response.set_index('date',drop=False)
    response.index.name = ''
    return response

# Add volume data to merged_df
print("Fetching and adding ETF volume data...")

assets = [
    'GBTC','BTC','IBIT','FBTC','ARKB',
    'BITB','BTCO','HODL','BRRR',
    'EZBC','BTCW'
]

volume_df = pd.DataFrame(columns=['date'])
count = 0

for i in assets:
    try:
        etf = stock_ticker(i)
        vol_name = i+'_Volume'
        price_name = i+'_Price'
        
        if count == 0:
            volume_df['date'] = etf['date']
            volume_df['AGG_Volume'] = 0
            
        volume_df[price_name] = etf['Close']
        volume_df[vol_name] = etf['Volume']*etf['Close']
        volume_df[vol_name] = volume_df[vol_name].fillna(0)
        volume_df['AGG_Volume'] = volume_df['AGG_Volume'] + volume_df[vol_name]
        count += 1
    except Exception as e:
        print(f"Error processing {i}: {str(e)}")
        continue

# Convert merged_df index to UTC timezone to match volume_df
merged_df.index = pd.to_datetime(merged_df.index).tz_localize('UTC')

# Merge with existing dataframe
merged_df = pd.concat([merged_df, volume_df.set_index('date')], axis=1)
merged_df = merged_df.fillna(0)
print("Updated columns in merged_df:", merged_df.columns.tolist())

Final columns in merged_df: ['price_usd_close', 'us_spot_etf_flows_net', 'us_spot_etf_balances_all_total', 'us_spot_etf_balances_all_GBTC', 'us_spot_etf_flows_all_total', 'us_spot_etf_flows_all_GBTC']
Fetching and adding ETF volume data...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GBTC']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['IBIT']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FBTC']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ARKB']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BITB']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***************

Updated columns in merged_df: ['price_usd_close', 'us_spot_etf_flows_net', 'us_spot_etf_balances_all_total', 'us_spot_etf_balances_all_GBTC', 'us_spot_etf_flows_all_total', 'us_spot_etf_flows_all_GBTC', 'AGG_Volume', 'GBTC_Price', 'GBTC_Volume', 'BTC_Price', 'BTC_Volume', 'IBIT_Price', 'IBIT_Volume', 'FBTC_Price', 'FBTC_Volume', 'ARKB_Price', 'ARKB_Volume', 'BITB_Price', 'BITB_Volume', 'BTCO_Price', 'BTCO_Volume', 'HODL_Price', 'HODL_Volume', 'BRRR_Price', 'BRRR_Volume', 'EZBC_Price', 'EZBC_Volume', 'BTCW_Price', 'BTCW_Volume']


In [55]:
volume_df

,date,AGG_Volume,GBTC_Price,GBTC_Volume,BTC_Price,BTC_Volume,IBIT_Price,IBIT_Volume,FBTC_Price,FBTC_Volume,...,BTCO_Price,BTCO_Volume,HODL_Price,HODL_Volume,BRRR_Price,BRRR_Volume,EZBC_Price,EZBC_Volume,BTCW_Price,BTCW_Volume


In [56]:
merged_df

,price_usd_close,us_spot_etf_flows_net,us_spot_etf_balances_all_total,us_spot_etf_balances_all_GBTC,us_spot_etf_flows_all_total,us_spot_etf_flows_all_GBTC,AGG_Volume,GBTC_Price,GBTC_Volume,BTC_Price,...,BTCO_Price,BTCO_Volume,HODL_Price,HODL_Volume,BRRR_Price,BRRR_Volume,EZBC_Price,EZBC_Volume,BTCW_Price,BTCW_Volume
2024-01-17 00:00:00+00:00,42730.926233,3.747317e+08,2.779013e+10,2.525151e+10,3.747317e+08,-5.876709e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-18 00:00:00+00:00,41261.242495,1.727958e+07,2.682575e+10,2.370797e+10,1.727958e+07,-4.447305e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-19 00:00:00+00:00,41609.562508,-4.745801e+08,2.704494e+10,2.353655e+10,-4.745801e+08,-6.726037e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-20 00:00:00+00:00,41648.718646,0.000000e+00,2.704494e+10,2.353655e+10,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-21 00:00:00+00:00,41535.759163,0.000000e+00,2.704494e+10,2.353655e+10,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-16 00:00:00+00:00,96157.604241,0.000000e+00,1.138550e+11,1.943767e+10,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-17 00:00:00+00:00,95837.336209,6.077715e+08,1.143890e+11,1.943767e+10,6.077715e+08,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-18 00:00:00+00:00,95671.896290,-1.757271e+09,1.107455e+11,1.943767e+10,-1.757271e+09,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-19 00:00:00+00:00,96685.073942,6.683878e+08,1.128533e+11,1.943767e+10,6.683878e+08,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
def calculate_smoothed_bands_v0(df, column = 'us_spot_etf_flows_net', smoothing_fun='sma', smoothing_window=1, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = df[column].rolling(window=7).sum()
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [58]:
def calculate_smoothed_bands_v1(df, columns=['us_spot_etf_balances_all_total','us_spot_etf_balances_all_GBTC','us_spot_etf_flow_all_total','us_spot_etf_flow_all_GBTC'], smoothing_fun='sma', smoothing_window=1,range_method='maxmin', range_window=90):
    
    ref_date = "2024-01-17"
    start_date = "2024-01-27"
    
    # Calculate the product according to the formula
    product = ((df[columns[0]] - df[columns[1]] - (df[columns[0]] - df[columns[1]]).loc[ref_date]) / 
          (df[columns[2]].cumsum() - df[columns[3]].cumsum()))
    
    # Subset the data from the specified date
    product = product[product.index >= start_date]
    
    # Calculate smoothed version based on method using the product
    if smoothing_fun == 'sma':
        smoothed_raw = product.rolling(window=smoothing_window).mean()
    elif smoothing_fun == 'ema':
        smoothed_raw = product.ewm(span=smoothing_window).mean()
    elif smoothing_fun == 'med':
        smoothed_raw = product.rolling(window=smoothing_window).median()
    else:
        raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
    
    # Calculate bands based on smoothed data
    if range_method == 'maxmin':
        upper_band = smoothed_raw.rolling(window=range_window).max()
        lower_band = smoothed_raw.rolling(window=range_window).min()
    
    elif range_method == 'zscore':
        mean = smoothed_raw.rolling(window=range_window).mean()
        std = smoothed_raw.rolling(window=range_window).std()
        upper_band = mean + std
        lower_band = mean - std
    
    elif range_method == 'percentile':
        upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
        lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
    
    elif range_method == 'meandev':
        mean = smoothed_raw.rolling(window=range_window).mean()
        mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
        upper_band = mean + mean_deviation
        lower_band = mean - mean_deviation
    
    else:
        raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
    
    return smoothed_raw, upper_band, lower_band

In [59]:
merged_df['etfFlow_sma_meandev'], merged_df['etfFlow_upper'], merged_df['etfFlow_lower']= calculate_smoothed_bands_v0(merged_df, column='us_spot_etf_flows_net',smoothing_fun='sma', smoothing_window=1, range_method='meandev', range_window=30)

merged_df['etfmvrv_sma_meandev'], merged_df['etfmvrv_upper'], merged_df['etfmvrv_lower']= calculate_smoothed_bands_v1(merged_df, columns=['us_spot_etf_balances_all_total','us_spot_etf_balances_all_GBTC','us_spot_etf_flows_all_total','us_spot_etf_flows_all_GBTC'],smoothing_fun='sma', smoothing_window=1, range_method='meandev', range_window=30)

merged_df['etfvol_sma_meandev'], merged_df['etfvol_upper'], merged_df['etfvol_lower']= calculate_smoothed_bands_v0(merged_df, column='AGG_Volume',smoothing_fun='sma', smoothing_window=1, range_method='meandev', range_window=30)

In [60]:
def create_indicator_chart(merged_df, indicator_column, chart_title, bands="on", upper_lower=["fee_upper", "fee_lower"]):
   # Filter data for the last year
   one_year_ago = pd.Timestamp.now(tz='UTC') - timedelta(days=730)
   merged_df_last_year = merged_df[merged_df.index > one_year_ago]

   # Define colors
   GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
   BLUE_COLOR = 'rgb(0, 0, 255)'  # Blue
   GREEN_COLOR = 'rgb(0, 255, 0)'  # Green for upper band
   RED_COLOR = 'rgb(255, 0, 0)'    # Red for lower band

   # Create the visualization
   fig = make_subplots(specs=[[{"secondary_y": True}]])

   # Add price trace
   fig.add_trace(
       go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], 
                 name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
       secondary_y=False,
   )

   # Add indicator trace
   indicator = merged_df_last_year[indicator_column]
   fig.add_trace(
       go.Scatter(
           x=merged_df_last_year.index,
           y=indicator,
           name=indicator_column,
           line=dict(color=BLUE_COLOR, width=2),
           mode='lines'
       ),
       secondary_y=True,
   )

   # Add band traces if enabled
   if bands == "on":
       # Simply use the provided column names directly
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[0]], 
               name="Upper Band",
               line=dict(color=GREEN_COLOR, width=2, dash='dash'),
               mode='lines'
           ),
           secondary_y=True,
       )
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[1]], 
               name="Lower Band",
               line=dict(color=RED_COLOR, width=2, dash='dash'),
               mode='lines'
           ),
           secondary_y=True,
       )

   # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
   for month in [1, 4, 7, 10]:
       for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
           # Make the date timezone-aware
           date = pd.Timestamp(year=year, month=month, day=1, tz='UTC')
           if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
               fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

   # Function to format number to K, M, B
   def format_number(num):
       if abs(num) >= 1e9:
           return f"{num/1e9:.2f}B"
       elif abs(num) >= 1e6:
           return f"{num/1e6:.2f}M"
       elif abs(num) >= 1e3:
           return f"{num/1e3:.2f}K"
       else:
           return f"{num:.2f}"
   
   last_value = indicator.dropna().iloc[-1] if not indicator.dropna().empty else np.nan
   formatted_value = format_number(last_value)

   # Add annotation for the last value
   fig.add_annotation(
       x=0.95,  
       y=last_value*0.88,  
       xref="paper",
       yref="y2", 
       text=formatted_value,
       showarrow=False,
       font=dict(size=18, color=BLUE_COLOR),
       align="left",
       xanchor="left",
       yanchor="middle",
   )

   # Update layout
   fig.update_layout(
       title={
           'text': chart_title,
           'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
       },
       showlegend=False,  
       hovermode="x unified",
       plot_bgcolor='white',
       paper_bgcolor='white',
       font={'color': 'black', 'size': 14},
       width=1000,  
       height=450,  
   )

   # Update axes
   fig.update_xaxes(
       showgrid=False, 
       tickfont={'color': 'black', 'size': 14},
       zeroline=False,
       title_text=''  
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=False, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18)
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=True, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       side='right',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18),
       autorange=True
   )

   return fig

In [61]:
# Create charts for each indicator
indicators = [
    ('etfFlow_sma_meandev',"Bitcoin: US Spot ETF Netflow", "on", ["etfFlow_upper", "etfFlow_lower"]),
    ('etfmvrv_sma_meandev',"Bitcoin: US Spot ETF MVRV (Exc. GBTC)", "on", ["etfmvrv_upper", "etfmvrv_lower"]),
    ('etfvol_sma_meandev',"Bitcoin: US Spot ETF Trade Volume (Weekly)", "on", ["etfvol_upper", "etfvol_lower"])
]

for indicator, title,  bands, upper_lower in indicators:
    fig = create_indicator_chart(merged_df, indicator, title, bands, upper_lower)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [62]:
merged_df.to_csv('bitcoin_analysis_data_spotETF2.csv', index=True)

In [63]:
merged_df.tail()

,price_usd_close,us_spot_etf_flows_net,us_spot_etf_balances_all_total,us_spot_etf_balances_all_GBTC,us_spot_etf_flows_all_total,us_spot_etf_flows_all_GBTC,AGG_Volume,GBTC_Price,GBTC_Volume,BTC_Price,...,BTCW_Volume,etfFlow_sma_meandev,etfFlow_upper,etfFlow_lower,etfmvrv_sma_meandev,etfmvrv_upper,etfmvrv_lower,etfvol_sma_meandev,etfvol_upper,etfvol_lower
2025-02-16 00:00:00+00:00,96157.604241,0.000000e+00,1.138550e+11,1.943767e+10,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.960417e+08,1.918228e+09,-1.675039e+08,1.485331,1.616500,1.487856,0.0,0.0,0.0
2025-02-17 00:00:00+00:00,95837.336209,6.077715e+08,1.143890e+11,1.943767e+10,6.077715e+08,0.0,0.0,0.0,0.0,0.0,...,0.0,5.881887e+08,1.883339e+09,-1.596090e+08,1.479428,1.610527,1.484013,0.0,0.0,0.0
2025-02-18 00:00:00+00:00,95671.896290,-1.757271e+09,1.107455e+11,1.943767e+10,-1.757271e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.007530e+09,1.846892e+09,-2.565371e+08,1.462236,1.604471,1.479105,0.0,0.0,0.0
2025-02-19 00:00:00+00:00,96685.073942,6.683878e+08,1.128533e+11,1.943767e+10,6.683878e+08,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.874023e+08,1.688789e+09,-2.981429e+08,1.480655,1.598589,1.475485,0.0,0.0,0.0
2025-02-20 00:00:00+00:00,98356.253997,3.876743e+07,1.143006e+11,1.943767e+10,3.876743e+07,0.0,0.0,0.0,0.0,0.0,...,0.0,3.997109e+08,1.555861e+09,-3.006133e+08,1.503285,1.590317,1.473411,0.0,0.0,0.0


In [64]:
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Change the grey color to a darker shade
GREY_COLOR = 'rgba(70, 70, 70, 0.9)'  # Darker, more opaque grey

def normalize_value(value, min_val, max_val):
    range_val = max_val - min_val
    if range_val == 0:
        return 0
    normalized = (value - min_val) / range_val * 200 - 100
    return normalized

def format_number(number):
    """Format number with k, M, B suffixes"""
    abs_number = abs(number)
    if abs_number >= 1e9:
        return f"{number/1e9:.2f}B"
    elif abs_number >= 1e6:
        return f"{number/1e6:.2f}M"
    elif abs_number >= 1e3:
        return f"{number/1e3:.2f}k"
    else:
        return f"{number:.2f}"

def create_dot_plot(df, metrics_titles, days=90):
    start_date = df.index[-1] - timedelta(days=days)
    df_filtered = df[df.index >= start_date]
    
    fig = go.Figure()
    y_values = []
    
    for metric, title in metrics_titles.items():
        min_val = df_filtered[metric].min()
        max_val = df_filtered[metric].max()
        current_val = df_filtered[metric].iloc[-1]
        
        # Normalize values
        norm_min = -100
        norm_max = 100
        norm_current = normalize_value(current_val, min_val, max_val)
        
        y_values.append(title)
        
        # Add grey line (darker)
        fig.add_trace(go.Scatter(
            x=[norm_min, norm_max], y=[title, title], mode='lines',
            line=dict(color=GREY_COLOR, width=3),  # Using darker grey
            showlegend=False
        ))
        
        # Add min value (red filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_min], y=[title], mode='markers+text',
            marker=dict(color='red', size=16, symbol='circle'),
            text=[format_number(min_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Min'
        ))
        
        # Add max value (green filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_max], y=[title], mode='markers+text',
            marker=dict(color='green', size=16, symbol='circle'),
            text=[format_number(max_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Max'
        ))
        
        # Add current value (grey halo dot)
        fig.add_trace(go.Scatter(
            x=[norm_current], y=[title], mode='markers+text',
            marker=dict(
                color='white',
                size=16,
                line=dict(color=GREY_COLOR, width=3),  # Using darker grey
                symbol='circle'
            ),
            text=[format_number(current_val)],
            textposition="top center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Current'
        ))
    
    # Add vertical dashed lines at -50%, 0%, and 50%
    for value in [-50, 0, 50]:
        fig.add_shape(
            type="line",
            x0=value, x1=value, y0=0, y1=1,
            yref="paper",
            xref="x",
            line=dict(color=GREY_COLOR, width=1, dash="dash")  # Using darker grey
        )
    
    fig.update_layout(
        title={
            'text': f'Metric Values Over Last {days} Days',
            'font': {'color': GREY_COLOR, 'size': 20},  # Using darker grey
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={
            'text': 'Normalized Value (%)',
            'font': {'color': GREY_COLOR, 'size': 16}  # Using darker grey
        },
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_values))),
            ticktext=y_values,
            tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
            side='left',
            title_standoff=35
        ),
        height=400 + (len(metrics_titles) * 30),
        width=1200,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=100, r=100, t=50, b=20),
        font=dict(color=GREY_COLOR, size=14)  # Using darker grey
    )
    
    fig.update_xaxes(
        showgrid=False, 
        range=[-110, 110],
        tickvals=[-100, -50, 0, 50, 100],
        ticktext=['-100%<br>MIN-90D', '-50%', '0%', '50%', '100%<br>MAX-90D'],
        tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
        showline=False,
        zeroline=False
    )
    fig.update_yaxes(showgrid=False)
    
    return fig

In [65]:
# Manually define metrics and their titles
metrics_titles = {

    'etfFlow_sma_meandev': "US Spot ETF Netflow",
    'etfmvrv_sma_meandev': "US Spot ETF MVRV (Exc. GBTC)",
    'etfvol_sma_meandev': "US Spot ETF Volume (Weekly)"
       # Add more metrics and titles as needed
}

# Create the dot plot
fig = create_dot_plot(merged_df, metrics_titles)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='normalized_spotETF2_raw_metrics_dot_plot.html')